# Notebook for making grids of molecules (SI)

In [1]:
import pandas as pd

In [2]:
#Import all experiments to visualise
#The experiments chosen below are the ones selected for the paper

experiment_path = "experiments/"
experiment_names = ["exp_22", "exp_23", "exp_24", "exp_25", "exp_26", "exp_27", "exp_28"] # paper results for silvr rate
experiment_names += ["exp_33", "exp_13", "exp_26", "exp_10", "exp_18"] # visualisation results

#Paths to experiments
exp_array = [f"{experiment_path}{x}/" for x in experiment_names]


#Map each experiment with the fragments used in the reference
fragments_used = {
    "exp_10":["Mpro-x0874_0A","Mpro-x0397_0A"],
    "exp_18":["Mpro-x0874_0A","Mpro-x0397_0A"],
    "exp_13":["Mpro-x0434_0A","Mpro-x0305_0A"],
    "exp_15":["Mpro-x1093_0A","Mpro-x0072_0A", "Mpro-x2193_0A"],
    "exp_32":["Mpro-x1093_0A","Mpro-x0072_0A", "Mpro-x2193_0A"],
    "exp_33":["Mpro-x1093_0A","Mpro-x0072_0A", "Mpro-x2193_0A"],
    "exp_26":["Mpro-x0072_0A", "Mpro-x0354_0A"],
    "rates":["Mpro-x0072_0A", "Mpro-x0354_0A"]
}

In [3]:
import pandas as pd
import glob

def xyz_to_pd(xyz_path):
    """
    All information from sample XYZ file is parsed into an array
    
    Input (string): file path to XYZ of a molecule generated by SILVR
    
    Output (list): mol_id, xyz, stable_atoms, total_atoms, stable_ratio
        mol_id : name of molecule (molecules are named by their timestamp and GPU)
        xyz : the XYZ data from the sampled molecule
        stable_atoms : the number of stable atoms, determined by bond lengths
        total_atoms : the total number of atoms in the molecule
        stable_ratio : stable_atoms / total_atoms - the ratio of atoms stable by bond lengths
    
    
    """
    mol_id = xyz_path.split("/")[-1].split(".")[0]
    
    with open(xyz_path, "r") as readfile:
        file = readfile.read().split("\n")        
        data = file[1].split(" ")
        
        if data[0].split(":")[1]=="True":
            stable = True
        else:
            stable = True
    
        stable_atoms = int(data[1].split(":")[1])
        total_atoms = int(data[2].split(":")[1])
        stable_ratio = float(data[3].split(":")[1])
        xyz = "\n".join(file)
        
    return [mol_id, xyz, stable_atoms, total_atoms, stable_ratio]

def reference_to_pd(dir_path):
    """
    All information from experiment design reference.sxyz file is parsed and returned as list
    
    input (string path): PATH TO EXPERIMENT DIRECTORY. Note: All experiments MUST constain the reference
        file "reference.sxyz" which defined the full experiment protocol
    
    output (list): [exp_id, xyz, reference_atoms, dummy_atoms, samples, silvr, comment]
        exp_id : name of experiment (given by directory)
        xyz : XYZ coordinates of reference
        reference_atoms : total number of atoms in the reference file
        dummy_atoms : This is not described within the SILVR paper but should be contained in the SI. 
            Dummy atoms are atoms which are added when running the diffusion model (EDM) however are NOT
            MAPPED to a reference atom. That is, the dummy atoms are free to fly around and fit wherever they like.
            This is of particular importance for linker design type generation.
        samples : How many samples to run
        silvr : The silvr rate used in the experiment. If a single value was used for all atoms, the returned value here
            is a single float, however if any of the values are different, this value returns a numpy array. This is
            used when running "selective" experiments, where different atoms recieve a different silvr value depending 
            on priority
        comment : any additional information about the experiment design
    """
    ref_path = dir_path+"reference.sxyz"
    exp_id = ref_path.split("/")[-2]
    
    with open(ref_path,"r") as readfile:
        file = readfile.read().split("\n")
        reference_atoms = int(file[0])
        
        data = file[1].split(" ")
        dummy_atoms = int(data[0].split(":")[1])
        samples = int(data[1].split(":")[1])
        comment = " ".join(data[2:]).split(":")[1]
        
        
        xyz = "\n".join(file)
        
    silvr_data = pd.read_csv(ref_path, delim_whitespace=True, skiprows=2, names=["Element", "X", "Y", "Z", "SILVR"])
    silvr_vector = silvr_data["SILVR"].to_numpy()
    if (silvr_vector[0] == silvr_vector).all():
        silvr = silvr_vector[0]
    else:
        silvr = silvr_vector
    
    #If all values in the array are the same, then just return a float
    
    return [exp_id, xyz, reference_atoms, dummy_atoms, samples, silvr, comment]

In [4]:
#---Import all experiment details into exp_df dataframe------------
#exp_df["exp_id"] is used as a primary key
exp_data = []
for exp in exp_array:
    exp_data.append(reference_to_pd(exp))
exp_df = pd.DataFrame(exp_data,columns=["exp_id", "ref_xyz", "reference_atoms", "dummy_atoms", "samples", "silvr", "comment"])
exp_df.set_index("exp_id")

#---Import all results into results_df----
#results_df["exp_id"] is a foreign key and references exp_df["exp_id"]
results_list = []
for exp in exp_array:
    exp_id = exp.split("/")[-2]
    for xyz in glob.glob(f"{exp}mol_*.txt"):
        results_list.append(xyz_to_pd(xyz)+[exp_id])

results_df = pd.DataFrame(results_list,columns=["mol_id", "xyz", "stable_atoms", "total_atoms", "stable_ratio","exp_id"])
results_df.set_index("mol_id")

#---Merge tables on ["exp_id"]---
#The dataframe df contains all experiment information from xyz and reference.sxyz files
df = pd.merge(results_df,exp_df, on="exp_id")

In [5]:
from rdkit.Chem import Descriptors
from rdkit.Chem.MolStandardize import rdMolStandardize
from openbabel import pybel
from rdkit import Chem


def rdkit_fix_radicals(mol,add_h=False,flatten=False,uncharge=True):
    """
    Atoms with unfilled valance get radicals assigned.
    Openbabel will have assigned bond orders based on bond length.
    Here I assume all radical electrons should instead be hydrogen atoms
    """
    for atom in mol.GetAtoms():
        radicals = atom.GetNumRadicalElectrons()
        atom.SetNumRadicalElectrons(0)
        atom.SetNumExplicitHs(atom.GetNumExplicitHs() + radicals)

    if flatten:
        mol = Chem.MolFromSmiles(Chem.MolToSmiles(mol))

    if add_h:
        mol = Chem.AddHs(mol,addCoords=True)

    if uncharge:
        un = rdMolStandardize.Uncharger()
        mol = un.uncharge(mol)

    return mol


def xyz_to_mol_clean(xyz, add_h=True, flatten=False):
    """
    add_h - add RDKit hydrogens
    flatten - run Chem.MolFromSmiles(Chem.MolToSmiles(x)) such that geometry infromation is lost
    
    Sometimes these imports fail
    In these cases this function returns False
    """
    try:
        mol_pybel = pybel.readstring("xyz", xyz)
        mol_mol2 = mol_pybel.write("mol2")

        #RDKit - clean radicals
        mol_rdkit = Chem.MolFromMol2Block(mol_mol2)
        mol_final = rdkit_fix_radicals(mol_rdkit, add_h=add_h, flatten=flatten)

        return mol_final
    
    except:
        return False

In [6]:
df['mol'] = df['xyz'].apply(xyz_to_mol_clean)

[19:37:08] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:22 tatoms:31 sratio:0.7096774193548387)

[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Explicit valence for atom # 13 C, 5, is greater than permitted
[19:37:08] sanitise [19:37:08] stable:False satoms:26 tatoms:31 sratio:0.8387096774193549: [19:37:08] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:25 tatoms:31 sratio:0.8064516129032258)

[19:37:08] Running Uncharger
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:27 tatoms:31 sratio

[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:25 tatoms:31 sratio:0.8064516129032258)

[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08]

[19:37:08] Running Uncharger
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:27 tatoms:31 sratio:0.8709677419354839)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:22 tatoms:31 sratio:0.7096774193548387)

*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:21 tatoms:31 sratio:0.6774193548387096)

[19:37:08] Running Uncharger
[19:37:08] Removed negative charge.
[19:37:08] Running Uncharger
[19:37

[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Can't kekulize mol.  Unkekulized atoms: 6 8 10 13 14
[19:37:09] sanitise [19:37:09] stable:False satoms:29 tatoms:31 sratio:0.9354838709677419: [19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (titl

[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
*** Open Babel Warning  in PerceiveBondOrders
  Failed to kekulize aromatic bonds in OBMol::PerceiveBondOrders (title is stable:False satoms:25 tatoms:31 sratio:0.8064516129032258)

[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Removed negative charge.
[19:37:09] Running Uncharger
[19:37:09] Remov

...

[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running Uncharger
[19:38:35] Removed negative charge.
[19:38:35] Removed negative charge.
[19:38:35] Running 

[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed 

[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:3

[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Can't kekulize mol.  Unkekulized atoms: 10 11 12 17 20
[19:38:36] sanitise [19:38:36] stable:False satoms:27 tatoms:38 sratio:0.7105263157894737: [19:38:36] Running Uncharger
[19:38:36] Removed negative charge.
[19:38:36] Removed negative charge.
[19:38:36] Running Uncharger

[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] Running Uncharger
[19:38:37] Removed negative charge.
[19:38:37] Removed negative charge.
[19:38:37] R

# Visualise Mpro ligands

In [7]:
from rdkit.Chem import Draw

mpro_ligands = Chem.SDMolSupplier("mpro_ligands/Mpro_combined.sdf")
mpro_ligands_2d = [Chem.MolFromSmiles(Chem.MolToSmiles(x)) for x in mpro_ligands]
ligand_names = [x.GetProp('_Name') for x in mpro_ligands]

img = Draw.MolsToGridImage(mpro_ligands_2d, molsPerRow=4,
                     #useSVG=True,
                     subImgSize=(250,250),
                     legends=ligand_names,
                     #useSVG=True,
                     returnPNG=True
                    )

png = img.data
with open('mol_grids/mpro_ligands_grid_2d.png','wb+') as outf:
    outf.write(png)

# Visualise grids for each experiment (not currated)

In [38]:
#Select n=molsPerRow molecules from each rate
from rdkit.Chem import rdmolops

molsPerRow = 4
n_rows = 6
mols_grid_array = []
legends = []


for exp in experiment_names:
    if exp in ["exp_22", "exp_23", "exp_24", "exp_25", "exp_26", "exp_27", "exp_28"]:
        reference_names = fragments_used["rates"]
    else:
        reference_names = fragments_used[exp]
    reference_mols = []
    silvr_rate = str(exp_df[exp_df["exp_id"]==exp]["silvr"].iloc[0])
    
    for name in reference_names:
        idx = ligand_names.index(name)
        reference_mols.append(mpro_ligands_2d[idx])
    
    row_0 =  [Chem.MolFromSmiles("*")]*(molsPerRow - len(reference_names)) + reference_mols
    legends_0 = [f"{exp}      SILVR = {silvr_rate}"] + [""]*(molsPerRow - len(reference_names) - 1) + reference_names

    mols_grid_array = row_0 + [Chem.MolFromSmiles(Chem.MolToSmiles(rdmolops.RemoveAllHs(x))) for x in list(df[(df["mol"]!=False) & (df["exp_id"]==exp)].iloc[:molsPerRow*n_rows]["mol"])]
    legends = legends_0 + [""]*n_rows*molsPerRow
    
    img = Draw.MolsToGridImage(mols_grid_array, molsPerRow=molsPerRow,
                         #useSVG=True,
                         subImgSize=(300,300),
                         legends=legends,
                         #useSVG=True,
                         returnPNG=True
                        )

    png = img.data
    with open(f'mol_grids/mols_grid_{exp}.png','wb+') as outf:
        outf.write(png)

# Show SILVR rates grid (not currated)

In [26]:
#Select n=molsPerRow molecules from each rate
molsPerRow = 5

#Sort by QED
#Only select non fragmented
#Make 2D

#There is probably a better way to make this array
mols_grid_array = []
legends = []

from rdkit.Chem import rdmolops

reference_names = fragments_used["rates"]
reference_mols = []
for name in reference_names:
    idx = ligand_names.index(name)
    reference_mols.append(mpro_ligands_2d[idx])

mols_grid_array += reference_mols + [Chem.MolFromSmiles("*")]*(molsPerRow - len(reference_names))
legends += reference_names + [""]*(molsPerRow - len(reference_names))


for exp in ["exp_22", "exp_23", "exp_24", "exp_25", "exp_26", "exp_27", "exp_28"]:
    #For flat stucture use Chem.MolFromSmiles(Chem.MolToSmiles(x))
    #For no rdmolops.RemoveAllHs(x)
    #For molecule as is us x
    selection_list = list(df[(df["exp_id"]==exp) & df["mol"]!= False].iloc[:molsPerRow]["mol"])
    selection = [Chem.MolFromSmiles(Chem.MolToSmiles(x)) for x in selection_list]
    
    mols_grid_array += selection
    
    legends += ["SILVR = " + str(exp_df[exp_df["exp_id"]==exp]["silvr"].iloc[0])]*molsPerRow
    
img = Draw.MolsToGridImage(mols_grid_array, molsPerRow=molsPerRow,
                     #useSVG=True,
                     subImgSize=(250,250),
                     legends=legends
                    )

png = img.data
with open(f'mol_grids/mols_grid_silvr_rate_not_curated.png','wb+') as outf:
    outf.write(png)

# Show SILVR rate grid (currated)

Selection criteria. Ignore fragmented molecules. Take the 200 best scoring molecules by synthetic accessibility, then select the 100 best by qed (drug likeness). This subset should represent the most drug like and synthetically viable molecules. 
`df[(df["is_fragmented"]==False) & (df["exp_id"]==exp)].nsmallest(200, "sa").nlargest(100, "qed")`

In [27]:
from rdkit.Chem import Draw
from rdkit.Chem import rdmolops
from rdkit import Chem

def is_fragmented(mol, tolerance=None):
    """
    Check if the molecule has been fragmented
    That is, within the RDkit molecule, there are two or more disconnected molecules
    
    Input (RDKit mol): must be RDKit mol object
    
    tolerance (int): can be defined to account for small benign fragments like water
        Molecule is split into fragments. Fragments are a tuple of tuple, with indexes refering to which atoms
        are in which fragment. The length of the tuple indicates the number of atoms in the fragment.
        If the fragment size is larger than the tolerance, the tolerance_counter is incremented. In the case
        where only one fragment is large, and the rest are small, the tolerance_counter will equal 1. If the 
        tolerance_counter is not 1, this means the molecule is fragmented with large fragments. 
        
    Returns (bool): True when the molecule is fragmented
    """
    is_fragmented = False
    fragments = rdmolops.GetMolFrags(mol)
    
    if tolerance:
        tolerance_counter = 0
        for fragment in fragments:
            if len(fragment) > tolerance:
                tolerance_counter += 1
        if tolerance_counter != 1:
            is_fragmented = True
    else:
        if len(fragments) > 1:
            is_fragmented = True
            
    return is_fragmented

In [28]:
from rdkit.Chem import QED
df["qed"] = df[df["mol"]!=False]["mol"].apply(QED.qed)
df["is_fragmented"] = df[df["mol"]!=False]["mol"].apply(is_fragmented)

from SA_Score import sascorer
df['sa'] = df[df["is_fragmented"]==False]['mol'].apply(sascorer.calculateScore)

[19:48:09] WARNING: not removing hydrogen atom without neighbors
[19:48:10] WARNING: not removing hydrogen atom without neighbors
[19:48:11] WARNING: not removing hydrogen atom without neighbors
[19:48:12] WARNING: not removing hydrogen atom without neighbors
[19:48:12] WARNING: not removing hydrogen atom without neighbors
[19:48:13] WARNING: not removing hydrogen atom without neighbors


In [29]:
from rdkit.Chem import rdmolops

#Select n=molsPerRow molecules from each rate
molsPerRow = 5

#Sort by QED
#Only select non fragmented
#Make 2D

#There is probably a better way to make this array
mols_grid_array = []
legends = []

reference_names = fragments_used["rates"]
reference_mols = []
for name in reference_names:
    idx = ligand_names.index(name)
    reference_mols.append(mpro_ligands_2d[idx])


mols_grid_array += reference_mols + [Chem.MolFromSmiles("*")]*(molsPerRow - len(reference_names))
legends += reference_names + [""]*(molsPerRow - len(reference_names))

for exp in ["exp_22", "exp_23", "exp_24", "exp_25", "exp_26", "exp_27", "exp_28"]:
    #For flat stucture use Chem.MolFromSmiles(Chem.MolToSmiles(x))
    #For no rdmolops.RemoveAllHs(x)
    #For molecule as is us x
    selection = [Chem.MolFromSmiles(Chem.MolToSmiles(rdmolops.RemoveAllHs(x))) for x in list(df[(df["is_fragmented"]==False) & (df["exp_id"]==exp)].nsmallest(200, "sa").nlargest(100, "qed")["mol"].iloc[:molsPerRow])]
    mols_grid_array += selection
    
    legends += ["SILVR = " + str(exp_df[exp_df["exp_id"]==exp]["silvr"].iloc[0])]*molsPerRow

    
img = Draw.MolsToGridImage(mols_grid_array, molsPerRow=molsPerRow,
                     #useSVG=True,
                     subImgSize=(250,250),
                     legends=legends
                    )

png = img.data
with open(f'mol_grids/mols_grid_silvr_rate_curated.png','wb+') as outf:
    outf.write(png)